In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd


In [2]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor



In [4]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 465kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.19MB/s]


In [6]:
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [7]:
train_loader = DataLoader(train_data, batch_size=100, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=100, shuffle=True, pin_memory=True)

In [8]:
class MyModel(nn.Module):
  def __init__(self, input):
    super().__init__()

    self.feature = nn.Sequential(
        nn.Conv2d(input,32, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32,64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        )
    self.fatten = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*5*5, 128),
        nn.ReLU(),
        nn.Linear(128, 10),

    )

  def forward(self, x):
      x = self.feature(x)
      x = self.fatten(x)
      return x

In [9]:
learing_rate = 0.01
epochs = 50

In [10]:
model = MyModel(1)
model.to(dev)


loss_fun = nn.CrossEntropyLoss()


optimzer = torch.optim.Adam(model.parameters(), lr = learing_rate)

In [11]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features , batch_labels in train_loader:


    batch_features, batch_labels = batch_features.to(dev), batch_labels.to(dev)

    y_pred = model(batch_features)

    loss = loss_fun(y_pred, batch_labels)

    optimzer.zero_grad()
    loss.backward()

    optimzer.step()

    total_epoch_loss = total_epoch_loss + loss.item()


  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 0.19701075855991804
Epoch: 2 , Loss: 0.075391854494034
Epoch: 3 , Loss: 0.05921153584514589
Epoch: 4 , Loss: 0.05463026766859305
Epoch: 5 , Loss: 0.05301904306203748
Epoch: 6 , Loss: 0.047314093823855125
Epoch: 7 , Loss: 0.04674969558963009
Epoch: 8 , Loss: 0.04539799733873224
Epoch: 9 , Loss: 0.04795470004561745
Epoch: 10 , Loss: 0.041524045846830025
Epoch: 11 , Loss: 0.04258162146851343
Epoch: 12 , Loss: 0.03909011687222422
Epoch: 13 , Loss: 0.043740056774319476
Epoch: 14 , Loss: 0.04305460499474672
Epoch: 15 , Loss: 0.03694696896498499
Epoch: 16 , Loss: 0.04146967159961302
Epoch: 17 , Loss: 0.035443476076907246
Epoch: 18 , Loss: 0.03628364689420475
Epoch: 19 , Loss: 0.03711639672608726
Epoch: 20 , Loss: 0.03746414192912198
Epoch: 21 , Loss: 0.03474313504188709
Epoch: 22 , Loss: 0.03669156719542722
Epoch: 23 , Loss: 0.03757235517895121
Epoch: 24 , Loss: 0.03015321321384666
Epoch: 25 , Loss: 0.03940421966302286
Epoch: 26 , Loss: 0.02959604455379425
Epoch: 27 , Loss: 0

In [12]:
model.eval()

MyModel(
  (feature): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fatten): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1600, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [13]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(dev), batch_labels.to(dev)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9778


In [14]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(dev), batch_labels.to(dev)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9894666666666667


In [15]:
Model_path = 'model.pth'
torch.save(model.state_dict(), Model_path)